In [1]:
# imports

import os
import requests
from bs4 import BeautifulSoup
import json
from typing import List
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import google.generativeai
import anthropic

In [2]:
import gradio as gr # oh yeah!

In [3]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyCi


In [4]:
# Connect to OpenAI, Anthropic

openai = OpenAI()

claude = anthropic.Anthropic()

#google.generativeai.configure()

### Brochure generator

The following code is for an application taking in a webpage and generating a brochure for it.
The Brochure for a company can be used for prospective clients, investors and potential recruits.

Code steps:
1. class Website: a class representing a web page with a title, body and the list of links
2. function get_links(url): using LLM to extract the links that would be relevant for a brochure in a json format:
        a) system prompt with one-shot encoding making explicit how the output should look like
        b) function get_links_user_prompt(website): user prompt providing that instruction and list of scraped links
5. function get_all_details(url): creating the Website object, and returning the contents of the url + all relevant links (= long block of text)
6. function create_brochure(company_name, url): using llm to create a brochure based on all raw contents:
        a) system prompt explaining what the brochure should look like
        b) function get_brochure_user_prompt(company_name, url): user prompt providing all the raw contents
7. Use gradio interface to run function create_brochure(company_name, url)

### Class website

In [5]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [ ]:
adc = Website("https://adc-consulting.com/")
adc.links

### Function get_links

In [12]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [ ]:
print(link_system_prompt)

In [14]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [19]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [22]:
get_links("https://adc-consulting.com/")

{'links': [{'type': 'about page',
   'url': 'https://adc-consulting.com/about-us/'},
  {'type': 'careers page', 'url': 'https://adc-consulting.com/careers/'},
  {'type': 'services page', 'url': 'https://adc-consulting.com/services/'},
  {'type': 'industries page', 'url': 'https://adc-consulting.com/industries/'},
  {'type': 'contact page', 'url': 'https://adc-consulting.com/contact/'},
  {'type': 'cases page', 'url': 'https://adc-consulting.com/cases/'},
  {'type': 'news page', 'url': 'https://adc-consulting.com/news/'}]}

### Function get_all_details

In [23]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [ ]:
print(get_all_details("https://adc-consulting.com/"))

### Function create_brochure

In [26]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

In [27]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [ ]:
get_brochure_user_prompt("Amsterdam Data Collective", "https://adc-consulting.com/")

In [48]:
def stream_brochure_gpt(company_name, url):
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [49]:
def stream_brochure_claude(company_name, url):
    result = claude.messages.stream(
        model="claude-3-haiku-20240307",
        max_tokens=1000,
        temperature=0.7,
        system=system_prompt,
        messages=[
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)},
        ],
    )
    response = ""
    with result as stream:
        for text in stream.text_stream:
            response += text or ""
            yield response

### Call Gradio

In [36]:
force_dark_mode = """
function refresh() {
    const url = new URL(window.location);
    if (url.searchParams.get('__theme') !== 'dark') {
        url.searchParams.set('__theme', 'dark');
        window.location.href = url.href;
    }
}
"""

In [50]:
def stream_brochure_model(company_name, url, model):
    if model=="GPT":
        result = stream_brochure_gpt(company_name, url)
    elif model=="Claude":
        result = stream_brochure_claude(company_name, url)
    else:
        raise ValueError("Unknown model")
    yield from result

In [51]:
view = gr.Interface(
    fn=stream_brochure_model,
    inputs=[gr.Textbox(label="Company name:"), gr.Textbox(label="Company url"), gr.Dropdown(["GPT", "Claude"], label="Select model")],
    outputs=[gr.Markdown(label="Brochure:")],
    flagging_mode="never", js=force_dark_mode
)
view.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7878

To create a public link, set `share=True` in `launch()`.


Found links: {'links': [{'type': 'about page', 'url': 'https://adc-consulting.com/about-us/'}, {'type': 'careers page', 'url': 'https://adc-consulting.com/careers/'}, {'type': 'services page', 'url': 'https://adc-consulting.com/services/'}, {'type': 'industries page', 'url': 'https://adc-consulting.com/industries/'}, {'type': 'cases page', 'url': 'https://adc-consulting.com/cases/'}, {'type': 'news page', 'url': 'https://adc-consulting.com/news/'}, {'type': 'contact page', 'url': 'https://adc-consulting.com/contact/'}]}
Found links: {'links': [{'type': 'about page', 'url': 'https://adc-consulting.com/about-us/'}, {'type': 'careers page', 'url': 'https://adc-consulting.com/careers/'}, {'type': 'services page', 'url': 'https://adc-consulting.com/services/'}, {'type': 'industries page', 'url': 'https://adc-consulting.com/industries/'}, {'type': 'cases page', 'url': 'https://adc-consulting.com/cases/'}, {'type': 'news page', 'url': 'https://adc-consulting.com/news/'}, {'type': 'contact pag